In [1]:
import ee
import geemap

# ee.Authenticate()
ee.Initialize(project="ee-zuzannaslobodzian2")

In [2]:
polygon = geemap.shp_to_ee("C:\\space_technologies\\OpenPitMining\\niedzwedzia_gora\\niedzwiedzia_gora.shp")
image = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterBounds(polygon).filterDate('2025-03-01', '2025-03-31').sort('CLOUDY_PIXEL_PERCENTAGE').first()

polygon

In [3]:
rgb = image.select(['B4', 'B3', 'B2'])

image = image.clip(polygon.first().geometry().buffer(1000))
ndvi = image.normalizedDifference(['B8', 'B4'])

palette_ndvi = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
palette_rgb = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}
centre = polygon.first().geometry().centroid(maxError=1).coordinates().getInfo()

map = geemap.Map(center=[centre[1], centre[0]], zoom=14)

map.add_layer(rgb, palette_rgb, "ortofotomapa")
map.add_layer(ndvi, palette_ndvi, "NDVI")
map.add_layer(polygon, {}, "obszar górniczy")
map

Map(center=[50.10339412565508, 19.63649860680545], controls=(WidgetControl(options=['position', 'transparent_b…